In [1]:
import numpy as np
import tensorflow as tf

/Users/brianregan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
n_boat = 15
n_anchor = 9
img_size = 5
batch_size = 10

In [3]:
# Simpler Case 
# Array Creation
# main array is a [batch_size, h, w, n_boat, 4] shaped array where values are repeated along the last dimension.
# in the first slice of the n_boat dimension each array is batch_index*[[0,1,2,3,4], [1,2,3,4,5], ...] while the second
# slice in the n_boat dimension is batch_index*[[[1,2,3,4,5], [2,3,4,5,6]] ...] (ie. shifted up one).
np.random.seed(seed=0)

batch_arrs = []
for b in range(batch_size):
    arrs = []
    for i in range(n_boat):
        arrs.append(np.arange(start=i, stop=i +img_size*img_size).reshape((img_size, img_size)))
    
    batch_array = np.stack(arrs, -1)
    batch_array = np.repeat(np.expand_dims(batch_array, -1), 4, -1)
    #main_array = np.repeat(np.expand_dims(main_array, 0), batch_size, 0)
    batch_arrs.append(batch_array*b)

main_array = np.stack(batch_arrs, 0)
    

index_array = np.random.randint(n_boat, size=(img_size, img_size))
index_array = np.repeat(np.expand_dims(index_array, 0), batch_size, 0)

In [4]:
# [batch_size, h, w, n_boat, 4]
main_array.shape

(10, 5, 5, 15, 4)

In [5]:
# [batch_size, h, w]
index_array.shape

(10, 5, 5)

In [6]:
main_array[1,:,:,2, 0]

array([[ 2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21],
       [22, 23, 24, 25, 26]])

In [7]:
index_array[3,:,:]

array([[12,  5,  0,  3, 11],
       [ 3,  7,  9,  3,  5],
       [ 2,  4,  7,  6,  8],
       [ 8, 12, 10,  1,  6],
       [ 7,  7, 14,  8,  1]])

In [8]:
# See https://stackoverflow.com/questions/28980345/numpy-index-array-by-array
out = main_array[np.arange(batch_size)[:,np.newaxis, np.newaxis], 
                 np.arange(img_size)[np.newaxis, :, np.newaxis], 
                 np.arange(img_size)[np.newaxis, np.newaxis, :], 
                 index_array]



In [9]:
def select_with_matrix(arr, indexer):
    '''
    
    '''
    
    batch_size, h, w, depth, vals = arr.shape
    out = arr[np.arange(batch_size)[:,np.newaxis, np.newaxis], 
              np.arange(h)[np.newaxis, :, np.newaxis], 
              np.arange(w)[np.newaxis, np.newaxis, :], 
              indexer]
    return out

In [50]:
def select_with_matrix_tf(tensor, indexer):
    
    batch_size, h, w, depth, vals = tensor.get_shape().as_list()
    desired_shape = [batch_size, h, w]
    index_list = [
        tf.broadcast_to(tf.reshape(tf.range(batch_size, dtype=tf.int64), (-1, 1, 1)), desired_shape),
        tf.broadcast_to(tf.reshape(tf.range(h, dtype=tf.int64), (1,-1,1)), desired_shape),
        tf.broadcast_to(tf.reshape(tf.range(w, dtype=tf.int64), (1,1,-1)), desired_shape),
        indexer
    ]
    index = tf.stack(index_list,-1)
    return tf.gather_nd(tensor, index)

In [51]:
main_array_tf, index_array_tf = tf.constant(main_array), tf.constant(index_array, dtype=tf.int64)
out_tf = select_with_matrix_tf(main_array_tf, index_array_tf)

In [17]:
sess = tf.InteractiveSession()

In [41]:
out_tf_eval = out_tf.eval()

In [26]:
out = select_with_matrix(main_array, index_array)

In [27]:
out.shape

(10, 5, 5, 4)

In [31]:
out[1, :,:,1]

array([[12,  6,  2,  6, 15],
       [ 8, 13, 16, 11, 14],
       [12, 15, 19, 19, 22],
       [23, 28, 27, 19, 25],
       [27, 28, 36, 31, 25]])

In [42]:
out_tf_eval[1, :,:,1]

array([[12,  6,  2,  6, 15],
       [ 8, 13, 16, 11, 14],
       [12, 15, 19, 19, 22],
       [23, 28, 27, 19, 25],
       [27, 28, 36, 31, 25]])

In [47]:
dif = (out - out_tf_eval)
print(dif.max(), dif.min(), np.median(dif))

0 0 0.0


In [73]:
a = out_tf[:, :,:,0]

In [74]:
a.shape

TensorShape([Dimension(10), Dimension(5), Dimension(5)])

In [89]:
b = tf.tile(tf.expand_dims(a, -1), [1,1,1,4]).eval()

In [92]:
b[:,:,:,2]

array([[[  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0]],

       [[ 12,   6,   2,   6,  15],
        [  8,  13,  16,  11,  14],
        [ 12,  15,  19,  19,  22],
        [ 23,  28,  27,  19,  25],
        [ 27,  28,  36,  31,  25]],

       [[ 24,  12,   4,  12,  30],
        [ 16,  26,  32,  22,  28],
        [ 24,  30,  38,  38,  44],
        [ 46,  56,  54,  38,  50],
        [ 54,  56,  72,  62,  50]],

       [[ 36,  18,   6,  18,  45],
        [ 24,  39,  48,  33,  42],
        [ 36,  45,  57,  57,  66],
        [ 69,  84,  81,  57,  75],
        [ 81,  84, 108,  93,  75]],

       [[ 48,  24,   8,  24,  60],
        [ 32,  52,  64,  44,  56],
        [ 48,  60,  76,  76,  88],
        [ 92, 112, 108,  76, 100],
        [108, 112, 144, 124, 100]],

       [[ 60,  30,  10,  30,  75],
        [ 40,  65,  80,  55,  70],
        [ 60,  75,  95,  95, 110],
        [1